In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import os
import glob
import csv

Using TensorFlow backend.


In [118]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'GLOBAL', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [165]:
def new_dataset(dataset, step_size):
    data_X, data_Y = [], []
    for i in range(len(dataset)-step_size-1):
        a = dataset[i:(i+step_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + step_size, 0])
    return np.array(data_X), np.array(data_Y)

In [260]:
path =r'C:\Users\asus\Desktop\Stock_LSTM\SET50'
allFiles = glob.glob(path + "/*.csv")
allName = os.listdir(path)
d = {os.path.basename(f).split('.')[0]:pd.read_csv(f,usecols=[4]).dropna().reset_index(drop=True).round(1) for f in allFiles if "" in f}

In [261]:
scaler = MinMaxScaler(feature_range=(0, 1))
train , test , data , target = [] , [] , [] , []
for key, value in d.items():
    d[key] = np.reshape(value.values, (len(value),1))
    d[key] = scaler.fit_transform(value)
    train.append(int(len(d[key])*0.8))
    test.append(len(d[key]) - int(len(d[key])*0.8))
    data.append(d[key][0:int(len(d[key])*0.8),:])
    target.append(d[key][int(len(d[key])*0.8):len(d[key]),:])

In [262]:
x_train , y_train , x_test , y_test = [] , [] , [] , []
for value in data:
    x_train.append(new_dataset(value,1))
    y_train.append(new_dataset(value,1))
for value in target:
    x_test.append(new_dataset(value,1))
    y_test.append(new_dataset(value,1))

In [263]:
for index, value in enumerate(x_train):
    x_train[index] = np.reshape(x_train[index][0], (x_train[index][0].shape[0], 1, x_train[index][0].shape[1]))
for index, value in enumerate(x_test):
    x_test[index] = np.reshape(x_test[index][0], (x_test[index][0].shape[0], 1, x_test[index][0].shape[1]))

In [278]:
print('Shape of x_train')
for index, value in enumerate(x_train):
    print(x_train[index].shape)
print('\nShape of  x_test')
for index, value in enumerate(x_test):
    print(x_test[index].shape)

Shape of x_train
(3362, 1, 1)
(2503, 1, 1)
(3536, 1, 1)
(3386, 1, 1)
(4324, 1, 1)
(2623, 1, 1)
(2146, 1, 1)
(2504, 1, 1)
(2662, 1, 1)
(2320, 1, 1)
(2638, 1, 1)
(1253, 1, 1)
(3342, 1, 1)
(1613, 1, 1)
(1852, 1, 1)
(3391, 1, 1)
(1577, 1, 1)
(3143, 1, 1)
(1325, 1, 1)
(3006, 1, 1)
(3394, 1, 1)
(2407, 1, 1)
(3284, 1, 1)
(1677, 1, 1)
(2554, 1, 1)
(3145, 1, 1)
(2972, 1, 1)
(1566, 1, 1)
(2334, 1, 1)
(3392, 1, 1)
(3311, 1, 1)
(3342, 1, 1)
(1569, 1, 1)
(3387, 1, 1)
(2374, 1, 1)
(3536, 1, 1)
(811, 1, 1)

Shape of  x_test
(840, 1, 1)
(625, 1, 1)
(883, 1, 1)
(846, 1, 1)
(1080, 1, 1)
(655, 1, 1)
(535, 1, 1)
(625, 1, 1)
(665, 1, 1)
(579, 1, 1)
(659, 1, 1)
(312, 1, 1)
(835, 1, 1)
(402, 1, 1)
(462, 1, 1)
(847, 1, 1)
(393, 1, 1)
(785, 1, 1)
(330, 1, 1)
(750, 1, 1)
(847, 1, 1)
(601, 1, 1)
(820, 1, 1)
(418, 1, 1)
(637, 1, 1)
(785, 1, 1)
(742, 1, 1)
(390, 1, 1)
(582, 1, 1)
(847, 1, 1)
(827, 1, 1)
(835, 1, 1)
(391, 1, 1)
(846, 1, 1)
(592, 1, 1)
(883, 1, 1)
(202, 1, 1)


In [282]:
model = Sequential()
model.add(LSTM(32, input_shape=(1, step_size), return_sequences = True))
model.add(LSTM(16))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='adagrad',metrics=['acc' , 'mse' ]) 
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test) , batch_size=1, verbose=2 )

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 37 arrays: [array([[[ 0.05661713]],

       [[ 0.04670913]],

       [[ 0.03963199]],

       ..., 
       [[ 0.87968861]],

       [[ 0.89738146]],

       [[ 0.90092003]]]), array([[[ 0.10973578]],

       [[ ...